Dataset link

https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_datasets.html

In [1]:
!python --version

Python 3.10.18


In [2]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2025-01-30")
census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

# for convenience, indexing on the soma_joinid which links this to other census data.
census_datasets = census_datasets.set_index("soma_joinid")

census_datasets

,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,,,,
0,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,4eb29386-de81-452f-b3c0-e00844e8c7fd,f76861bb-becb-4eb7-82fc-782dc96ccc7f,Spatial transcriptomics in mouse: Puck_191112_05,4eb29386-de81-452f-b3c0-e00844e8c7fd.h5ad,10888
1,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,78d59e4a-82eb-4a61-a1dc-da974d7ea54b,7d7ec1b6-6e3f-4aaa-9442-4b22f3424396,Spatial transcriptomics in mouse: Puck_191112_08,78d59e4a-82eb-4a61-a1dc-da974d7ea54b.h5ad,10250
2,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,add5eb84-5fc9-4f01-982e-a346dd42ee82,de54aed8-4f73-48f6-9229-418a840e2d82,Spatial transcriptomics in mouse: Puck_191109_20,add5eb84-5fc9-4f01-982e-a346dd42ee82.h5ad,12906
3,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,b020294c-ab82-4547-b5a7-63d8ffa575ed,abe4fce1-0859-4a56-ad1e-734d79f0e6c8,Spatial transcriptomics in mouse: Puck_191112_13,b020294c-ab82-4547-b5a7-63d8ffa575ed.h5ad,15161
4,Publication: https://doi.org/10.1038/s41591-02...,a96133de-e951-4e2d-ace6-59db8b3bfb1d,HTAN/HTAPP Broad - Spatio-molecular dissection...,10.1038/s41591-024-03215-z,Klughammer et al. (2024) Nat Med,d7476ae2-e320-4703-8304-da5c42627e71,863fc5e4-bd4a-4681-9c3d-0ee7ef54e327,HTAPP-330-SMP-1082 scRNA-seq,d7476ae2-e320-4703-8304-da5c42627e71.h5ad,565
...,...,...,...,...,...,...,...,...,...,...
1568,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfa2614-7ca7-4d82-814c-350626eccb26,3002a659-a1a9-4406-9976-99e658e1fbb5,Major cell cluster: Mesoderm,dcfa2614-7ca7-4d82-814c-350626eccb26.h5ad,3267338
1569,Publication: https://doi.org/10.1126/science.a...,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,The Tabula Sapiens Consortium* et al. (2022) S...,53d208b0-2cfd-4366-9866-c3c6114081bc,10df7690-6d10-4029-a47e-0f071bb2df83,Tabula Sapiens - All Cells,53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad,1136218
1570,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3,3817734b-0f82-433b-8c38-55b214200fff,Whole dataset: Raw counts only,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3.h5ad,11441407


In [6]:
def format_description_prompt(row):
    prompt = f"""
Please summarize the following single-cell RNA-seq dataset entry clearly and concisely in 100–150 words, using plain paragraph format.

Focus on:
- The tissue or organ studied
- Organism
- Number of cells and samples (if available)
- Sample conditions such as health status, age, sex, or strain
- Experimental method (e.g. Slide-seqV2)
- Study purpose and biological context (e.g. disease studied, pathways analyzed)
- Major annotated or observed cell types

Avoid listing raw metadata fields or repeating links. Do **not** include generic downstream analysis or embedding use cases.
This summary should be metadata-rich and biologically informative.

Dataset Title: {row['dataset_title']}
Citation: {row['citation']}
DOI: {row['collection_doi']} ({row['collection_doi_label']})
Collection Name: {row['collection_name']} (ID: {row['collection_id']})
Cell Count: {row['dataset_total_cell_count']:,} cells

Describe this dataset in natural language:
""".strip()
    return prompt


In [7]:
import requests, os, json

def openrouter_chat(model: str, prompt: str):
    resp = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
            "Content-Type": "application/json"
        },
        data=json.dumps({
            "model": model,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    return resp.json()

# Example usage of the OpenRouter API to summarize a dataset description
model = "deepseek/deepseek-r1-0528:free"
query = format_description_prompt(census_datasets.iloc[0])
print("User: ")
print(query)

result = openrouter_chat(model, query)
message = result["choices"][0]["message"]["content"]
print(f"\nAssistant ({model}): ")
print(message)

User: 
Please summarize the following single-cell RNA-seq dataset entry clearly and concisely in 100–150 words, using plain paragraph format.

Focus on:
- The tissue or organ studied
- Organism
- Number of cells and samples (if available)
- Sample conditions such as health status, age, sex, or strain
- Experimental method (e.g. Slide-seqV2)
- Study purpose and biological context (e.g. disease studied, pathways analyzed)
- Major annotated or observed cell types

Avoid listing raw metadata fields or repeating links. Do **not** include generic downstream analysis or embedding use cases.
This summary should be metadata-rich and biologically informative.

Dataset Title: Spatial transcriptomics in mouse: Puck_191112_05
Citation: Publication: https://doi.org/10.1016/j.isci.2022.104097 Dataset Version: https://datasets.cellxgene.cziscience.com/f76861bb-becb-4eb7-82fc-782dc96ccc7f.h5ad curated and distributed by CZ CELLxGENE Discover in Collection: https://cellxgene.cziscience.com/collections/8

In [ ]:
import pandas as pd

# The census_datasets DataFrame is large, this is a subset for demonstration
df = census_datasets[0:10]

def generate_description(row, model="deepseek/deepseek-r1-0528-qwen3-8b:free"):
    prompt = format_description_prompt(row)
    result = openrouter_chat(model, prompt)
    try:
        return result["choices"][0]["message"]["content"].strip()
    except (KeyError, IndexError) as e:
        return f"[Error retrieving description: {e}]"

new_df = pd.DataFrame({
    'dataset_title': df['dataset_title'],
    'collection_doi': df['collection_doi'],
    'dataset_total_cell_count': df['dataset_total_cell_count'],
    'description': df.apply(generate_description, axis=1)
})

new_df.head()